In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Tobi-Bueck/customer-support-tickets")

# Convert to pandas DataFrame for easier manipulation in Kaggle
df = dataset['train'].to_pandas()

# Display first few rows
df.head()

README.md: 0.00B [00:00, ?B/s]

aa_dataset-tickets-multi-lang-5-2-50-ver(…):   0%|          | 0.00/26.0M [00:00<?, ?B/s]

dataset-tickets-multi-lang-4-20k.csv:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/48587 [00:00<?, ? examples/s]

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51.0,Security,Outage,Disruption,Data Breach,None,None,None,None
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51.0,Account,Disruption,Outage,IT,Tech Support,None,None,None
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51.0,Product,Feature,Tech Support,None,None,None,None,None
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51.0,Billing,Payment,Account,Documentation,Feedback,None,None,None
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51.0,Product,Feature,Feedback,Tech Support,None,None,None,None


In [3]:
df_copy = df.copy()

In [4]:
df_copy.shape

(48587, 16)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming df is your loaded DataFrame
print("Initial shape:", df_copy.shape)
print("\nColumn types:\n", df_copy.dtypes)
print("\nMissing values:\n", df_copy.isnull().sum())

Initial shape: (48587, 16)

Column types:
 subject      object
body         object
answer       object
type         object
queue        object
priority     object
language     object
version     float64
tag_1        object
tag_2        object
tag_3        object
tag_4        object
tag_5        object
tag_6        object
tag_7        object
tag_8        object
dtype: object

Missing values:
 subject      5299
body            2
answer         11
type            0
queue           0
priority        0
language        0
version     20000
tag_1           0
tag_2          59
tag_3         231
tag_4        4597
tag_5       20951
tag_6       35362
tag_7       42619
tag_8       46115
dtype: int64


In [6]:

essential_cols = [None,'subject','body','type','queue','priority','answer','language','version','tag_1', 'tag_2', 'tag_3', 'tag_4', 
    'tag_5', 'tag_6', 'tag_7', 'tag_8']


df_processed = df_copy

# Add synthetic ticket ID for traceability
df_processed['ticket_id'] = range(1, len(df)+1)

# Create combined text field
df_processed['full_text'] = df_processed['subject'] + ' ' + df_processed['body']

# Convert all tag columns to string type and combine
tag_cols = [f'tag_{i}' for i in range(1,9)]
df_processed[tag_cols] = df_processed[tag_cols].fillna('').astype(str)
df_processed['all_tags'] = df_processed[tag_cols].apply(lambda x: '|'.join(x), axis=1)

# Create preprocessing notes
df_processed['preprocessing_notes'] = ''

# Display new structure
print("Processed columns:", df_processed.columns.tolist())
print("\nSample record:")
display(df_processed.iloc[0])

Processed columns: ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8', 'ticket_id', 'full_text', 'all_tags', 'preprocessing_notes']

Sample record:


subject                                  Wesentlicher Sicherheitsvorfall
body                   Sehr geehrtes Support-Team,\n\nich möchte eine...
answer                 Vielen Dank für die Meldung des kritischen Sic...
type                                                            Incident
queue                                                  Technical Support
priority                                                            high
language                                                              de
version                                                             51.0
tag_1                                                           Security
tag_2                                                             Outage
tag_3                                                         Disruption
tag_4                                                        Data Breach
tag_5                                                                   
tag_6                                              

In [7]:
# Create missing value flags before imputation
missing_cols = ['subject', 'body', 'answer', 'version', 'tag_2', 'tag_3', 'tag_4']
for col in missing_cols:
    df_processed[f'{col}_missing'] = df_processed[col].isnull().astype(int)

# 1. Text Columns
df_processed['subject'].fillna('No Subject', inplace=True)
df_processed['body'].fillna('', inplace=True)  # Empty string better than 'No Body' for NLP
df_processed['answer'].fillna('No Resolution', inplace=True)

# 2. Version - Create special category
df_processed['version'].fillna('Unknown Version', inplace=True)

# 3. Tags - Progressive filling strategy
df_processed['tag_2'].fillna('', inplace=True)
df_processed['tag_3'].fillna(df_processed['tag_2'], inplace=True)  # Fallback to tag_2
df_processed['tag_4'].fillna(df_processed['tag_3'], inplace=True)  # Fallback to tag_3

# 4. High-missing tags (5-8) - Binary indicators
for i in range(5,9):
    col = f'tag_{i}'
    df_processed[col] = df_processed[col].notnull().astype(int)
    df_processed.rename(columns={col: f'has_{col}'}, inplace=True)

# Verify treatment
print("Missing values after treatment:")
print(df_processed[[
    'subject', 'body', 'answer', 'version', 
    'tag_2', 'tag_3', 'tag_4',
    'has_tag_5', 'has_tag_6', 'has_tag_7', 'has_tag_8'
]].isnull().sum())

Missing values after treatment:
subject      0
body         0
answer       0
version      0
tag_2        0
tag_3        0
tag_4        0
has_tag_5    0
has_tag_6    0
has_tag_7    0
has_tag_8    0
dtype: int64


/tmp/ipykernel_36/2089901248.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['subject'].fillna('No Subject', inplace=True)
/tmp/ipykernel_36/2089901248.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [8]:
# Extract version components 
df_processed['version_major'] = df_processed['version'].str.extract(r'(\d+)\.')[0]
df_processed['version_minor'] = df_processed['version'].str.extract(r'\.(\d+)')[0]

# Fill extracted components
df_processed['version_major'].fillna(-1, inplace=True)
df_processed['version_minor'].fillna(-1, inplace=True)

# Convert to numeric
df_processed['version_major'] = pd.to_numeric(df_processed['version_major'])
df_processed['version_minor'] = pd.to_numeric(df_processed['version_minor'])

/tmp/ipykernel_36/2989184632.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['version_major'].fillna(-1, inplace=True)
/tmp/ipykernel_36/2989184632.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_processed['version_major'].fillna(-1, inplace=True)
/tmp/ipykernel_36/2989184632.py:7: Futur

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

# 1. Create list of tag lists
tag_list = (
    df_processed['tag_2'].astype(str) + ',' + 
    df_processed['tag_3'].astype(str) + ',' + 
    df_processed['tag_4'].astype(str)
).str.split(',').apply(lambda x: [t for t in x if t != 'nan'])

# 2. Binarize frequent tags
mlb = MultiLabelBinarizer()
tag_matrix = mlb.fit_transform(tag_list)
frequent_tags = [tag for tag in mlb.classes_ if tag_matrix[:,mlb.classes_.tolist().index(tag)].sum() > 100]

for tag in frequent_tags:
    df_processed[f'tag_{tag}'] = tag_list.apply(lambda x: tag in x).astype(int)

In [10]:
# A. Text Feature Extraction (unchanged, correct)
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine cleaned text fields
df_processed['final_text'] = df_processed['subject'] + " " + df_processed['body']

# TF-IDF Vectorization (limit features for Kaggle's memory)
tfidf = TfidfVectorizer(max_features=500, stop_words='english')
text_features = tfidf.fit_transform(df_processed['final_text'])

# B. ALTERNATIVE for Time Features (since no created_at exists)
# Option 1: Use row order as proxy for ticket age if dataset is time-ordered
df_processed['ticket_sequence'] = range(len(df_processed))  # Creates 0,1,2,... as proxy

# Option 2: Create synthetic timestamps if you know the time period covered
# Example: Assuming tickets span 1 year
import numpy as np
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')
df_processed['created_at'] = pd.to_datetime(np.random.uniform(
    start_date.value, end_date.value, len(df_processed)
))

# C. Categorical Feature Encoding (unchanged, correct)
df_processed['priority'] = pd.Categorical(df_processed['priority'], 
                                        categories=['low','medium','high','critical'],
                                        ordered=True)
df_processed['priority_code'] = df_processed['priority'].cat.codes

# D. Tag Features (unchanged, correct)
tag_cols = [col for col in df_processed.columns if col.startswith('tag_')]

/tmp/ipykernel_36/496977160.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_processed['final_text'] = df_processed['subject'] + " " + df_processed['body']
/tmp/ipykernel_36/496977160.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_processed['ticket_sequence'] = range(len(df_processed))  # Creates 0,1,2,... as proxy
/tmp/ipykernel_36/496977160.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [11]:
# Modified SLA checker using sequence numbers instead of timestamps
def check_slas(ticket_queue):
    stale_tickets = []
    for i, ticket in ticket_queue.iterrows():
        if ticket['status'] == 'open' and ticket['ticket_sequence'] < (len(df_processed) - 1000):  # Example threshold
            stale_tickets.append(ticket['ticket_id'])
    return stale_tickets

In [12]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib



In [13]:
# 1. Data Preparation
df_processed['priority_code'] = pd.to_numeric(df_processed['priority_code'], errors='coerce').fillna(1)
df_processed['desc_length'] = df_processed['final_text'].str.len()
df_processed['tech_flag'] = df_processed['final_text'].str.contains('error|bug|fail', case=False).astype(int)

# 2. Feature Selection
X_text = df_processed['final_text']
X_numeric = df_processed[['priority_code', 'desc_length', 'tech_flag']].astype(float)
y = df_processed['queue']

# 3. Train-test split
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(
    X_text, X_numeric, y, test_size=0.2, random_state=42
)

# 4. Text processing
text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=500))
])
X_train_text_features = text_pipeline.fit_transform(X_train_text)

# 5. Combine features
X_train_final = np.hstack([
    X_train_text_features.toarray(),
    X_train_num.values
])

# 6. Train model
clf = RandomForestClassifier()
clf.fit(X_train_final, y_train)

# 7. Evaluate
X_test_final = np.hstack([
    text_pipeline.transform(X_test_text).toarray(),
    X_test_num.values
])
print(classification_report(y_test, clf.predict(X_test_final)))

/tmp/ipykernel_36/4142558437.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_processed['desc_length'] = df_processed['final_text'].str.len()
/tmp/ipykernel_36/4142558437.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_processed['tech_flag'] = df_processed['final_text'].str.contains('error|bug|fail', case=False).astype(int)


                                 precision    recall  f1-score   support

           Billing and Payments       0.91      0.67      0.77      1023
               Customer Service       0.59      0.69      0.64      1464
                General Inquiry       1.00      0.35      0.52       134
                Human Resources       1.00      0.46      0.63       194
                     IT Support       0.92      0.48      0.64      1121
                Product Support       0.69      0.61      0.65      1795
          Returns and Exchanges       0.98      0.49      0.66       487
            Sales and Pre-Sales       0.99      0.46      0.62       292
Service Outages and Maintenance       0.94      0.51      0.66       371
              Technical Support       0.59      0.91      0.72      2837

                       accuracy                           0.68      9718
                      macro avg       0.86      0.56      0.65      9718
                   weighted avg       0.74      0

In [14]:
# Save all necessary components
import joblib
from datetime import datetime

# Save model with timestamp
model_version = datetime.now().strftime("%Y%m%d_%H%M")
joblib.dump(text_pipeline, f'text_pipeline_{model_version}.pkl')
joblib.dump(clf, f'ticket_classifier_{model_version}.pkl')

# Save feature metadata
import json
feature_metadata = {
    'required_features': {
        'text': 'final_text',
        'numerical': ['priority_code', 'desc_length', 'tech_flag'],
        'tags': [f'tag_{i}' for i in range(1,5)]
    },
    'model_version': model_version,
    'training_date': datetime.now().isoformat()
}

with open(f'model_metadata_{model_version}.json', 'w') as f:
    json.dump(feature_metadata, f)

In [15]:
class TicketManager:
    def __init__(self):
        self.team_capacity = {
            'Technical Support': 10,
            'Customer Service': 8,
            # ... add all your teams
        }
        self.current_load = {team: 0 for team in self.team_capacity}
    
    def assign_ticket(self, ticket_data):
        # Preprocess
        features = self._prepare_features(ticket_data)
        
        # Predict
        predicted_team = clf.predict(features)[0]
        
        # Load balance
        if self.current_load[predicted_team] < self.team_capacity[predicted_team]:
            self.current_load[predicted_team] += 1
            return predicted_team
        else:
            return self._find_alternative_team(predicted_team)
    
    def _prepare_features(self, ticket_data):
        # Same preprocessing as training
        text_features = text_pipeline.transform([ticket_data.get('final_text', '')])
        num_features = [
            ticket_data.get('priority_code', 1),
            len(ticket_data.get('final_text', '')),
            int('error' in ticket_data.get('final_text', '').lower())
        ]
        return np.hstack([text_features.toarray(), [num_features]])
    
    def _find_alternative_team(self, preferred_team):
        # Find team with most similar skills and available capacity
        for team in sorted(self.team_capacity, 
                         key=lambda x: (self.current_load[x]/self.team_capacity[x], 
                                      x != preferred_team)):
            if self.current_load[team] < self.team_capacity[team]:
                self.current_load[team] += 1
                return team
        return "Unassigned"

# Initialize
manager = TicketManager()

In [16]:
class SLAMonitor:
    def __init__(self):
        self.tickets = {}  # {ticket_id: {'assigned_time': datetime, 'status': str}}
    
    def log_ticket(self, ticket_id, team):
        self.tickets[ticket_id] = {
            'assigned_time': datetime.now(),
            'status': 'assigned',
            'team': team
        }
    
    def check_violations(self):
        violations = []
        for ticket_id, details in self.tickets.items():
            if details['status'] == 'assigned':
                age = (datetime.now() - details['assigned_time']).total_seconds() / 3600
                if age > 24:  # 24 hour SLA
                    violations.append(ticket_id)
        return violations
    
    def update_status(self, ticket_id, status):
        if ticket_id in self.tickets:
            self.tickets[ticket_id]['status'] = status

# Initialize
sla_monitor = SLAMonitor()

In [17]:
class Notifier:
    @staticmethod
    def send_assignment(ticket_id, team):
        print(f"📬 Email to {team}: New ticket #{ticket_id} assigned")
    
    @staticmethod
    def send_sla_alert(ticket_id, team):
        print(f"⚠️ Email to {team}: Ticket #{ticket_id} is overdue!")
    
    @staticmethod
    def send_resolution(ticket_id, customer_email="user@example.com"):
        print(f"📩 Email to {customer_email}: Your ticket #{ticket_id} has been resolved")

In [18]:
def process_new_ticket(ticket_data):
    # Generate ticket ID
    ticket_id = f"TKT-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # 1. Assign to team
    assigned_team = manager.assign_ticket(ticket_data)
    sla_monitor.log_ticket(ticket_id, assigned_team)
    
    # 2. Send notification
    Notifier.send_assignment(ticket_id, assigned_team)
    
    # 3. Return response
    return {
        'ticket_id': ticket_id,
        'assigned_team': assigned_team,
        'timestamp': datetime.now().isoformat()
    }

# Example usage
new_ticket = {
    'final_text': "Cannot access my dashboard - 500 error",
    'priority_code': 2  # High priority
}

result = process_new_ticket(new_ticket)
print(f"\nCreated ticket: {result}")

📬 Email to Technical Support: New ticket #TKT-20250816-065317 assigned

Created ticket: {'ticket_id': 'TKT-20250816-065317', 'assigned_team': 'Technical Support', 'timestamp': '2025-08-16T06:53:17.961892'}


In [19]:
import time
from IPython.display import clear_output

def run_sla_checks():
    while True:
        clear_output(wait=True)
        stale_tickets = sla_monitor.check_violations()
        if stale_tickets:
            print(f"{len(stale_tickets)} overdue tickets detected!")
            for ticket in stale_tickets:
                team = sla_monitor.tickets[ticket]['team']
                Notifier.send_sla_alert(ticket, team)
        else:
            print("No SLA violations detected")
        time.sleep(3600)  # Check hourly

# Start in a separate thread (in Jupyter)
import threading
sla_thread = threading.Thread(target=run_sla_checks, daemon=True)
sla_thread.start()

No SLA violations detected


In [20]:
def resolve_ticket(ticket_id):
    if ticket_id in sla_monitor.tickets:
        sla_monitor.update_status(ticket_id, 'resolved')
        Notifier.send_resolution(ticket_id)
        manager.current_load[sla_monitor.tickets[ticket_id]['team']] -= 1
        return True
    return False

# Example resolution
resolve_ticket(result['ticket_id'])

📩 Email to user@example.com: Your ticket #TKT-20250816-065317 has been resolved


True

In [21]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Initialize data structures
if os.path.exists('tickets.csv'):
    tickets = pd.read_csv('tickets.csv', parse_dates=['created_at', 'resolved_at'])
else:
    tickets = pd.DataFrame(columns=[
        'ticket_id', 'subject', 'description', 'priority', 
        'assigned_team', 'status', 'created_at', 'resolved_at', 'resolution'
    ]).astype({
        'created_at': 'datetime64[ns]',
        'resolved_at': 'datetime64[ns]'
    })

team_capacity = {
    'Technical Support': 5,
    'Customer Service': 8,
    'Billing': 4,
    'General Inquiry': 6
}
team_load = {team: 0 for team in team_capacity}

if os.path.exists('audit_log.csv'):
    audit_log = pd.read_csv('audit_log.csv', parse_dates=['timestamp'])
else:
    audit_log = pd.DataFrame(columns=[
        'timestamp', 'ticket_id', 'action', 'user', 'details'
    ]).astype({'timestamp': 'datetime64[ns]'})

# Initialize models
def initialize_models():
    text_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=1000,
            ngram_range=(1,2),
            stop_words='english'
        ))
    ])
    
    clf = RandomForestClassifier(
        n_estimators=200,
        class_weight='balanced',
        max_depth=10,
        random_state=42
    )
    
    return text_pipeline, clf

def train_models(X_train, y_train):
    text_pipeline, clf = initialize_models()
    text_pipeline.fit(X_train['final_text'])
    text_features = text_pipeline.transform(X_train['final_text'])
    num_features = X_train[['priority_code', 'desc_length', 'tech_flag']].values
    X_train_final = np.hstack([text_features.toarray(), num_features])
    clf.fit(X_train_final, y_train)
    return text_pipeline, clf

# Train models with sample data
X_train = pd.DataFrame({
    'final_text': ["login error", "payment issue", "slow performance"],
    'priority_code': [2, 1, 3],
    'desc_length': [10, 13, 16],
    'tech_flag': [1, 0, 1]
})
y_train = np.array(["Technical Support", "Billing", "Technical Support"])

text_pipeline, clf = train_models(X_train, y_train)

# Helper functions
def predict_team(text, priority):
    text_feature = text_pipeline.transform([text])
    num_features = np.array([[priority, len(text), int('error' in text.lower())]])
    features = np.hstack([text_feature.toarray(), num_features])
    return clf.predict(features)[0]

def assign_ticket(predicted_team):
    global team_load
    
    if team_load[predicted_team] < team_capacity[predicted_team]:
        team_load[predicted_team] += 1
        return predicted_team
    
    for team in sorted(team_load, key=lambda x: team_load[x]/team_capacity[x]):
        if team_load[team] < team_capacity[team]:
            team_load[team] += 1
            return team
    return "Unassigned"

def save_data():
    tickets.to_csv('tickets.csv', index=False)
    audit_log.to_csv('audit_log.csv', index=False)
    print("Data saved successfully!")

# CLI Functions
def display_tickets():
    print("\n=== Current Tickets ===")
    if tickets.empty:
        print("No tickets available")
    else:
        print(tickets[['ticket_id', 'subject', 'priority', 'status', 'assigned_team']].to_string(index=False))

def create_ticket():
    global tickets, audit_log
    
    print("\n=== Create New Ticket ===")
    subject = input("Subject: ")
    description = input("Description: ")
    priority = input("Priority (Low/Medium/High/Critical): ").capitalize()
    
    if not description:
        print("Error: Description cannot be empty!")
        return
    
    new_ticket_id = f"TKT-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    predicted_team = predict_team(
        f"{subject} {description}", 
        ["Low", "Medium", "High", "Critical"].index(priority)
    )
    assigned_team = assign_ticket(predicted_team)
    
    new_ticket = {
        'ticket_id': new_ticket_id,
        'subject': subject,
        'description': description,
        'priority': priority,
        'assigned_team': assigned_team,
        'status': 'Open',
        'created_at': pd.to_datetime(datetime.now()),
        'resolved_at': pd.NaT,
        'resolution': None
    }
    
    tickets = pd.concat([tickets, pd.DataFrame([new_ticket])], ignore_index=True)
    
    # Log creation
    audit_log = pd.concat([
        audit_log,
        pd.DataFrame([{
            'timestamp': datetime.now(),
            'ticket_id': new_ticket_id,
            'action': 'creation',
            'user': 'system',
            'details': f"Assigned to {assigned_team}"
        }])
    ])
    
    print(f"\nTicket {new_ticket_id} created and assigned to {assigned_team}")
    save_data()

def process_tickets():
    global tickets, audit_log, team_load
    
    display_tickets()
    if tickets.empty:
        return
    
    print("\n=== Process Tickets ===")
    team = input("Enter your team name: ")
    
    team_tickets = tickets[
        (tickets['assigned_team'] == team) & 
        (tickets['status'] == 'Open')
    ]
    
    if team_tickets.empty:
        print(f"No open tickets assigned to {team}!")
        return
    
    print(f"\n=== {team}'s Open Tickets ===")
    print(team_tickets[['ticket_id', 'subject', 'priority']].to_string(index=False))
    
    ticket_id = input("\nEnter Ticket ID to resolve: ")
    if ticket_id not in team_tickets['ticket_id'].values:
        print("Invalid Ticket ID for your team!")
        return
    
    print(f"\nProcessing Ticket {ticket_id}")
    selected_ticket = tickets[tickets['ticket_id'] == ticket_id].iloc[0]
    print(f"\nSubject: {selected_ticket['subject']}")
    print(f"Priority: {selected_ticket['priority']}")
    print(f"Created: {selected_ticket['created_at'].strftime('%Y-%m-%d %H:%M')}")
    print(f"\nDescription:\n{selected_ticket['description']}")
    
    resolution = input("\nEnter resolution notes: ")
    if not resolution.strip():
        print("Resolution notes cannot be empty!")
        return
    
    idx = tickets.index[tickets['ticket_id'] == ticket_id].tolist()[0]
    tickets.at[idx, 'status'] = 'Resolved'
    tickets.at[idx, 'resolved_at'] = datetime.now()
    tickets.at[idx, 'resolution'] = resolution
    team_load[team] -= 1
    
    # Log resolution
    audit_log = pd.concat([
        audit_log,
        pd.DataFrame([{
            'timestamp': datetime.now(),
            'ticket_id': ticket_id,
            'action': 'resolution',
            'user': team,
            'details': f"Resolution: {resolution[:100]}..."
        }])
    ])
    
    print(f"\nTicket {ticket_id} resolved!")
    save_data()

def generate_reports():
    print("\n=== Reports Dashboard ===")
    
    # Ticket Status
    print("\n1. Ticket Status Summary:")
    status_counts = tickets['status'].value_counts()
    print(status_counts.to_string())
    
    # Team Workload
    print("\n2. Team Workload:")
    workload = pd.DataFrame({
        'Team': team_load.keys(),
        'Assigned': team_load.values(),
        'Capacity': team_capacity.values(),
        'Utilization %': [round(team_load[team]/team_capacity[team]*100, 1) for team in team_load]
    })
    print(workload.to_string(index=False))
    
    # Resolution Times
    resolved = tickets[tickets['status'] == 'Resolved']
    if not resolved.empty:
        resolved['resolution_time'] = (resolved['resolved_at'] - resolved['created_at']).dt.total_seconds()/3600
        print(f"\n3. Average Resolution Time: {resolved['resolution_time'].mean():.1f} hours")
    
    # Audit Log
    print("\n4. Recent Activity (last 5 actions):")
    print(audit_log.sort_values('timestamp', ascending=False).head(5).to_string(index=False))
    
    # Detailed Tickets View
    print("\n5. All Tickets:")
    print(tickets.sort_values('created_at', ascending=False).to_string(index=False))

# Main menu
def main():
    while True:
        print("\n=== Customer Support Ticket System ===")
        print("1. Create New Ticket")
        print("2. Process Tickets")
        print("3. View Reports")
        print("4. Exit System")
        
        choice = input("Enter your choice (1-4): ")
        
        if choice == '1':
            create_ticket()
        elif choice == '2':
            process_tickets()
        elif choice == '3':
            generate_reports()
        elif choice == '4':
            save_data()
            print("System shutdown. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1-4.")

if __name__ == "__main__":
    main()


=== Customer Support Ticket System ===
1. Create New Ticket
2. Process Tickets
3. View Reports
4. Exit System


Enter your choice (1-4):  3



=== Reports Dashboard ===

1. Ticket Status Summary:
Series([], )

2. Team Workload:
             Team  Assigned  Capacity  Utilization %
Technical Support         0         5            0.0
 Customer Service         0         8            0.0
          Billing         0         4            0.0
  General Inquiry         0         6            0.0

4. Recent Activity (last 5 actions):
Empty DataFrame
Columns: [timestamp, ticket_id, action, user, details]
Index: []

5. All Tickets:
Empty DataFrame
Columns: [ticket_id, subject, description, priority, assigned_team, status, created_at, resolved_at, resolution]
Index: []

=== Customer Support Ticket System ===
1. Create New Ticket
2. Process Tickets
3. View Reports
4. Exit System


Enter your choice (1-4):  4


Data saved successfully!
System shutdown. Goodbye!


In [22]:
users = {
    'admin': {'password': 'admin123', 'role': 'admin'},
    'support1': {'password': 'sup123', 'role': 'support', 'team': 'Technical Support'},
    'agent1': {'password': 'agent123', 'role': 'agent', 'team': 'Customer Service'}
}

current_user = None

def login():
    global current_user
    print("\n=== Login ===")
    username = input("Username: ")
    password = input("Password: ")
    
    if username in users and users[username]['password'] == password:
        current_user = username
        print(f"Welcome, {username}!")
        return True
    else:
        print("Invalid credentials")
        return False

In [23]:
categories = {
    'Technical': ['Login Issues', 'Performance', 'Error Messages'],
    'Billing': ['Payment', 'Refund', 'Invoice'],
    'General': ['Feedback', 'Complaint', 'Request']
}

def get_category():
    print("\nAvailable Categories:")
    for i, cat in enumerate(categories.keys(), 1):
        print(f"{i}. {cat}")
    choice = int(input("Select category (number): ")) - 1
    category = list(categories.keys())[choice]
    
    print("\nSubcategories:")
    for i, subcat in enumerate(categories[category], 1):
        print(f"{i}. {subcat}")
    sub_choice = int(input("Select subcategory (number): ")) - 1
    subcategory = categories[category][sub_choice]
    
    return f"{category} - {subcategory}"

In [24]:
def send_notification(ticket_id, action):
    print(f"\n[Email Notification] Ticket {ticket_id} has been {action}")
    # In a real system, you would integrate with an email service

In [25]:
def search_tickets():
    print("\n=== Search Tickets ===")
    print("1. By Ticket ID")
    print("2. By Status")
    print("3. By Team")
    print("4. By Date Range")
    choice = input("Search by: ")
    
    if choice == '1':
        ticket_id = input("Enter Ticket ID: ")
        result = tickets[tickets['ticket_id'] == ticket_id]
    elif choice == '2':
        status = input("Status (Open/Resolved): ").capitalize()
        result = tickets[tickets['status'] == status]
    elif choice == '3':
        team = input("Team name: ")
        result = tickets[tickets['assigned_team'] == team]
    elif choice == '4':
        start = input("Start date (YYYY-MM-DD): ")
        end = input("End date (YYYY-MM-DD): ")
        mask = (tickets['created_at'] >= start) & (tickets['created_at'] <= end)
        result = tickets[mask]
    
    print("\nSearch Results:")
    print(result.to_string(index=False) if not result.empty else "No tickets found")

In [26]:
def export_data():
    print("\n=== Export Data ===")
    print("1. Export Tickets")
    print("2. Export Audit Log")
    choice = input("Select option: ")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    if choice == '1':
        filename = f"tickets_export_{timestamp}.csv"
        tickets.to_csv(filename, index=False)
    elif choice == '2':
        filename = f"audit_log_export_{timestamp}.csv"
        audit_log.to_csv(filename, index=False)
    
    print(f"Data exported to {filename}")

In [27]:
def main_menu():
    while True:
        print("\n=== Customer Support Ticket System ===")
        print("1. Create New Ticket")
        print("2. Process Tickets")
        print("3. View Reports")
        print("4. Search Tickets")
        print("5. Export Data")
        print("6. System Settings")
        print("7. Exit System")
        
        choice = input("Enter your choice (1-7): ")
        
        if choice == '1':
            create_ticket()
        elif choice == '2':
            process_tickets()
        elif choice == '3':
            generate_reports()
        elif choice == '4':
            search_tickets()
        elif choice == '5':
            export_data()
        elif choice == '6':
            system_settings()
        elif choice == '7':
            save_data()
            print("System shutdown. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1-7.")

In [28]:
# Add to initialization
sla_rules = {
    'Critical': 1,    # 1 hour
    'High': 4,        # 4 hours
    'Medium': 24,     # 1 day
    'Low': 72         # 3 days
}

def check_sla_violations():
    now = datetime.now()
    violations = []
    
    for _, ticket in tickets[tickets['status'] == 'Open'].iterrows():
        hours_open = (now - ticket['created_at']).total_seconds() / 3600
        if hours_open > sla_rules[ticket['priority']]:
            violations.append(ticket['ticket_id'])
    
    if violations:
        print("\n⚠️ SLA VIOLATIONS DETECTED ⚠️")
        for ticket_id in violations:
            print(f"- {ticket_id} is overdue")
        # In production: Trigger alerts/emails

In [29]:
knowledge_base = {
    'login issues': "Try password reset at example.com/reset",
    'payment problems': "Contact billing at billing@example.com"
}

def search_knowledge_base():
    query = input("\nSearch knowledge base: ").lower()
    results = {k:v for k,v in knowledge_base.items() if query in k}
    
    if results:
        print("\nKnowledge Base Results:")
        for topic, solution in results.items():
            print(f"{topic.upper()}: {solution}")
    else:
        print("No matching articles found")

In [30]:
import matplotlib.pyplot as plt

def generate_visual_reports():
    # Resolution time distribution
    resolved = tickets[tickets['status'] == 'Resolved']
    if not resolved.empty:
        resolved['resolution_hours'] = (resolved['resolved_at'] - resolved['created_at']).dt.total_seconds() / 3600
        
        plt.figure(figsize=(10,5))
        plt.hist(resolved['resolution_hours'], bins=20)
        plt.title('Ticket Resolution Time Distribution')
        plt.xlabel('Hours')
        plt.ylabel('Number of Tickets')
        plt.savefig('resolution_times.png')
        print("\nSaved resolution time report to resolution_times.png")

In [31]:
def auto_prioritize(text):
    """AI-powered priority suggestion"""
    keywords = {
        'critical': ['down', 'outage', 'emergency'],
        'high': ['not working', 'broken', 'urgent'],
        'medium': ['question', 'help', 'issue'],
        'low': ['request', 'information']
    }
    
    text_lower = text.lower()
    for priority, terms in keywords.items():
        if any(term in text_lower for term in terms):
            return priority.capitalize()
    return 'Medium'  # Default

In [32]:
translations = {
    'es': {
        'menu': "=== Sistema de Tickets de Soporte ===",
        'options': ["1. Crear ticket", "2. Procesar tickets", "3. Salir"],
        'subject': "Asunto"
    },
    'fr': {
        'menu': "=== Système de Tickets d'Assistance ===",
        'options': ["1. Créer ticket", "2. Traiter tickets", "3. Quitter"],
        'subject': "Sujet"
    }
}

def set_language():
    print("\nAvailable languages:")
    print("1. English (en)")
    print("2. Español (es)")
    print("3. Français (fr)")
    choice = input("Select language: ")
    
    lang_map = {'1': 'en', '2': 'es', '3': 'fr'}
    return lang_map.get(choice, 'en')

In [ ]:
# ===== IMPORTS =====
import pandas as pd
import numpy as np
from datetime import datetime
import os
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# ===== INITIALIZATION =====
# Data files
TICKETS_FILE = 'tickets.csv'
AUDIT_FILE = 'audit_log.csv'
KNOWLEDGE_FILE = 'knowledge_base.json'

# Initialize data structures
def init_data():
    data = {}
    
    # Tickets
    if os.path.exists(TICKETS_FILE):
        data['tickets'] = pd.read_csv(TICKETS_FILE, parse_dates=['created_at', 'resolved_at'])
    else:
        data['tickets'] = pd.DataFrame(columns=[
            'ticket_id', 'subject', 'description', 'priority', 
            'assigned_team', 'status', 'created_at', 'resolved_at', 'resolution'
        ]).astype({
            'created_at': 'datetime64[ns]',
            'resolved_at': 'datetime64[ns]'
        })
    
    # Audit log
    if os.path.exists(AUDIT_FILE):
        data['audit_log'] = pd.read_csv(AUDIT_FILE, parse_dates=['timestamp'])
    else:
        data['audit_log'] = pd.DataFrame(columns=[
            'timestamp', 'ticket_id', 'action', 'user', 'details'
        ]).astype({'timestamp': 'datetime64[ns]'})
    
    # Knowledge base
    if os.path.exists(KNOWLEDGE_FILE):
        with open(KNOWLEDGE_FILE) as f:
            data['knowledge_base'] = json.load(f)
    else:
        data['knowledge_base'] = {
            'login issues': "Try password reset at example.com/reset",
            'payment problems': "Contact billing at billing@example.com"
        }
    
    return data

# ===== CORE CLASS =====
class TicketSystem:
    def __init__(self):
        self.data = init_data()
        self.team_capacity = {
            'Technical Support': 5,
            'Customer Service': 8,
            'Billing': 4,
            'General Inquiry': 6
        }
        self.team_load = {team: 0 for team in self.team_capacity}
        self.users = {
            'admin': {'password': 'admin123', 'role': 'admin', 'team': 'Admin'},
            'support1': {'password': 'sup123', 'role': 'support', 'team': 'Technical Support'},
            'agent1': {'password': 'agent123', 'role': 'agent', 'team': 'Customer Service'}
        }
        self.current_user = None
        self.init_models()
    
    def init_models(self):
        self.text_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(
                max_features=1000,
                ngram_range=(1,2),
                stop_words='english'
            ))
        ])
        self.clf = RandomForestClassifier(
            n_estimators=200,
            class_weight='balanced',
            max_depth=10,
            random_state=42
        )
        self.train_models()
    
    def train_models(self):
        X_train = pd.DataFrame({
            'final_text': ["login error", "payment issue", "slow performance"],
            'priority_code': [2, 1, 3],
            'desc_length': [10, 13, 16],
            'tech_flag': [1, 0, 1]
        })
        y_train = np.array(["Technical Support", "Billing", "Technical Support"])
        
        self.text_pipeline.fit(X_train['final_text'])
        text_features = self.text_pipeline.transform(X_train['final_text'])
        num_features = X_train[['priority_code', 'desc_length', 'tech_flag']].values
        X_train_final = np.hstack([text_features.toarray(), num_features])
        self.clf.fit(X_train_final, y_train)
    
    def predict_team(self, text, priority_code):
        text_feature = self.text_pipeline.transform([text])
        num_features = np.array([[priority_code, len(text), int('error' in text.lower())]])
        features = np.hstack([text_feature.toarray(), num_features])
        return self.clf.predict(features)[0]
    
    def assign_ticket(self, predicted_team):
        if self.team_load[predicted_team] < self.team_capacity[predicted_team]:
            self.team_load[predicted_team] += 1
            return predicted_team
        
        for team in sorted(self.team_load, key=lambda x: self.team_load[x]/self.team_capacity[x]):
            if self.team_load[team] < self.team_capacity[team]:
                self.team_load[team] += 1
                return team
        return "Unassigned"
    
    def save_data(self):
        self.data['tickets'].to_csv(TICKETS_FILE, index=False)
        self.data['audit_log'].to_csv(AUDIT_FILE, index=False)
        with open(KNOWLEDGE_FILE, 'w') as f:
            json.dump(self.data['knowledge_base'], f)
    
    def create_ticket(self):
        print("\n=== Create New Ticket ===")
        subject = input("Subject: ")
        description = input("Description: ")
        priority = input("Priority (Low/Medium/High/Critical): ").capitalize()
        
        if not description:
            print("Error: Description cannot be empty!")
            return
        
        new_ticket_id = f"TKT-{datetime.now().strftime('%Y%m%d%H%M%S')}"
        predicted_team = self.predict_team(
            f"{subject} {description}", 
            ["Low", "Medium", "High", "Critical"].index(priority)
        )
        assigned_team = self.assign_ticket(predicted_team)
        
        new_ticket = {
            'ticket_id': new_ticket_id,
            'subject': subject,
            'description': description,
            'priority': priority,
            'assigned_team': assigned_team,
            'status': 'Open',
            'created_at': pd.to_datetime(datetime.now()),
            'resolved_at': pd.NaT,
            'resolution': None
        }
        
        self.data['tickets'] = pd.concat([
            self.data['tickets'],
            pd.DataFrame([new_ticket])
        ], ignore_index=True)
        
        self.data['audit_log'] = pd.concat([
            self.data['audit_log'],
            pd.DataFrame([{
                'timestamp': datetime.now(),
                'ticket_id': new_ticket_id,
                'action': 'creation',
                'user': self.current_user,
                'details': f"Assigned to {assigned_team}"
            }])
        ])
        
        print(f"\nTicket {new_ticket_id} created and assigned to {assigned_team}")
        self.save_data()
    
    def process_tickets(self):
        # In your TicketSystem.__init__() method, update the users dictionary:
self.users = {
    'admin': {
        'password': 'admin123', 
        'role': 'admin',
        'team': 'Admin'  # Add this line
    },
    'support1': {
        'password': 'sup123', 
        'role': 'support', 
        'team': 'Technical Support'
    },
    'agent1': {
        'password': 'agent123', 
        'role': 'agent', 
        'team': 'Customer Service'
    }
}

# Then modify the process_tickets() method:
def process_tickets(self):
    print("\n=== Process Tickets ===")
    
    # Admin can process any team's tickets
    if self.users[self.current_user]['role'] == 'admin':
        print("Available teams:", list(self.team_capacity.keys()))
        team = input("Enter team to process tickets for: ")
        if team not in self.team_capacity:
            print("Invalid team!")
            return
    else:
        team = self.users[self.current_user]['team']
    
    team_tickets = self.data['tickets'][
        (self.data['tickets']['assigned_team'] == team) & 
        (self.data['tickets']['status'] == 'Open')
    ]
    
    if team_tickets.empty:
        print(f"No open tickets assigned to {team}!")
        return
    
    print(team_tickets[['ticket_id', 'subject', 'priority']].to_string(index=False))
    ticket_id = input("\nEnter Ticket ID to resolve: ")
    
    if ticket_id not in team_tickets['ticket_id'].values:
        print("Invalid Ticket ID!")
        return
        
    selected_ticket = self.data['tickets'][self.data['tickets']['ticket_id'] == ticket_id].iloc[0]
    print(f"\nSubject: {selected_ticket['subject']}")
    print(f"Priority: {selected_ticket['priority']}")
    print(f"Description:\n{selected_ticket['description']}")
    
    resolution = input("\nResolution notes: ")
    if not resolution.strip():
        print("Resolution notes cannot be empty!")
        return
        
    idx = self.data['tickets'].index[
        self.data['tickets']['ticket_id'] == ticket_id].tolist()[0]
    self.data['tickets'].at[idx, 'status'] = 'Resolved'
    self.data['tickets'].at[idx, 'resolved_at'] = datetime.now()
    self.data['tickets'].at[idx, 'resolution'] = resolution
    self.team_load[team] -= 1
    
    self.data['audit_log'] = pd.concat([
        self.data['audit_log'],
        pd.DataFrame([{
            'timestamp': datetime.now(),
            'ticket_id': ticket_id,
            'action': 'resolution',
            'user': self.current_user,
            'details': f"Resolution: {resolution[:100]}..."
        }])
    ])
    
    print(f"\nTicket {ticket_id} resolved!")
    self.save_data()
    
    def generate_reports(self):
        print("\n=== Reports ===")
        print("\n1. Ticket Status Summary:")
        print(self.data['tickets']['status'].value_counts().to_string())
        
        print("\n2. Team Workload:")
        workload = pd.DataFrame({
            'Team': self.team_load.keys(),
            'Assigned': self.team_load.values(),
            'Capacity': self.team_capacity.values(),
            'Utilization %': [round(self.team_load[t]/self.team_capacity[t]*100, 1) for t in self.team_load]
        })
        print(workload.to_string(index=False))
        
        resolved = self.data['tickets'][self.data['tickets']['status'] == 'Resolved']
        if not resolved.empty:
            resolved['resolution_time'] = (resolved['resolved_at'] - resolved['created_at']).dt.total_seconds()/3600
            print(f"\n3. Average Resolution Time: {resolved['resolution_time'].mean():.1f} hours")
        
        print("\n4. Recent Activity (last 5 actions):")
        print(self.data['audit_log'].tail(5).to_string(index=False))
        
        print("\n5. All Tickets:")
        print(self.data['tickets'].sort_values('created_at', ascending=False).to_string(index=False))
    
    def search_tickets(self):
        print("\n=== Search Tickets ===")
        print("1. By Ticket ID\n2. By Status\n3. By Team\n4. By Date Range")
        choice = input("Search by: ")
        
        if choice == '1':
            ticket_id = input("Ticket ID: ")
            result = self.data['tickets'][self.data['tickets']['ticket_id'] == ticket_id]
        elif choice == '2':
            status = input("Status (Open/Resolved): ").capitalize()
            result = self.data['tickets'][self.data['tickets']['status'] == status]
        elif choice == '3':
            team = input("Team name: ")
            result = self.data['tickets'][self.data['tickets']['assigned_team'] == team]
        elif choice == '4':
            start = input("Start date (YYYY-MM-DD): ")
            end = input("End date (YYYY-MM-DD): ")
            mask = (self.data['tickets']['created_at'] >= start) & (self.data['tickets']['created_at'] <= end)
            result = self.data['tickets'][mask]
        else:
            print("Invalid choice")
            return
            
        print("\nSearch Results:")
        print(result.to_string(index=False) if not result.empty else "No tickets found")
    
    def export_data(self):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.data['tickets'].to_csv(f"tickets_export_{timestamp}.csv", index=False)
        self.data['audit_log'].to_csv(f"audit_export_{timestamp}.csv", index=False)
        print(f"Data exported to tickets_export_{timestamp}.csv and audit_export_{timestamp}.csv")
    
    def system_tools(self):
        print("\n=== System Tools ===")
        print("1. Check SLA Compliance\n2. Rebuild Index\n3. Backup Data")
        choice = input("Select tool: ")
        if choice == '1':
            self.check_sla()
        else:
            print("Feature coming soon!")
    
    def check_sla(self):
        now = datetime.now()
        violations = self.data['tickets'][
            (self.data['tickets']['status'] == 'Open') &
            ((now - self.data['tickets']['created_at']).dt.total_seconds() > 
             self.data['tickets']['priority'].map({'Low':72, 'Medium':24, 'High':4, 'Critical':1})*3600)
        ]
        
        if not violations.empty:
            print("\n⚠️ SLA VIOLATIONS ⚠️")
            print(violations[['ticket_id', 'priority', 'created_at']])
        else:
            print("\nNo SLA violations detected")

# ===== MAIN EXECUTION =====
if __name__ == "__main__":
    system = TicketSystem()
    
    # Login
    while not system.current_user:
        print("\n=== Login ===")
        username = input("Username: ")
        password = input("Password: ")
        
        if username in system.users and system.users[username]['password'] == password:
            system.current_user = username
            print(f"Welcome, {username}!")
        else:
            print("Invalid credentials")
    
    # Main loop
    while True:
        print("\n=== Customer Support Ticket System ===")
        print("1. Create New Ticket")
        print("2. Process Tickets")
        print("3. View Reports")
        print("4. Search Tickets")
        print("5. Export Data")
        print("6. System Tools")
        print("7. Exit")
        
        choice = input("Enter choice (1-7): ")
        
        if choice == '1':
            system.create_ticket()
        elif choice == '2':
            system.process_tickets()
        elif choice == '3':
            system.generate_reports()
        elif choice == '4':
            system.search_tickets()
        elif choice == '5':
            system.export_data()
        elif choice == '6':
            system.system_tools()
        elif choice == '7':
            system.save_data()
            print("Goodbye!")
            break
        else:
            print("Invalid choice")